In [1]:
import pandas as pd 

In [3]:
# load data
off_street = pd.read_csv("off-street-car-parks-with-capacity-and-type.csv", low_memory=False)
off_street.head()

,Census year,Block ID,Property ID,Base property ID,Building address,CLUE small area,Parking type,Parking spaces,Longitude,Latitude,location
0,2007,521,110207,110207,41 Wolseley Parade KENSINGTON 3031,Kensington,Residential,1,144.928949,-37.796821,"-37.796820718883126, 144.92894902658225"
1,2007,522,102443,102443,13 Derby Street KENSINGTON 3031,Kensington,Residential,1,144.928087,-37.796539,"-37.796539134023334, 144.9280869152663"
2,2007,526,105238,105238,68-70 Kensington Road KENSINGTON 3031,Kensington,Residential,1,144.925053,-37.795359,"-37.795358984583245, 144.9250530930557"
3,2007,526,105248,105248,48 Kensington Road KENSINGTON 3031,Kensington,Residential,2,144.925698,-37.794986,"-37.79498560155301, 144.92569790198797"
4,2007,526,108278,108278,7-9 The Ridgeway KENSINGTON 3031,Kensington,Residential,2,144.926569,-37.794086,"-37.794086070069966, 144.92656869302766"


In [13]:
off_street.shape

(144014, 11)

In [5]:
df = off_street.copy()

In [9]:
df["Parking type"].unique()

array(['Residential', 'Private', 'Commercial'], dtype=object)

In [27]:
df["CLUE small area"].unique()

array(['Kensington', 'East Melbourne', 'Melbourne (Remainder)',
       'Southbank', 'Port Melbourne', 'South Yarra', 'Parkville',
       'West Melbourne (Industrial)', 'North Melbourne', 'Docklands',
       'Melbourne (CBD)', 'West Melbourne (Residential)', 'Carlton'],
      dtype=object)

In this onboarding project, we will only filtering the area of Melbourne CBD and some of the nearest location of cbd such as: 

- Docklands – major commercial and entertainment area
- Southbank – cultural precinct, high visitor numbers, multiple parking facilities within 5–15 min walk to CBD
- Carlton – north of CBD, home to Lygon Street restaurants, university buildings, and some cheaper parking options.

In [33]:
# Filter the dataset
filtered_df = (
    off_street[
        (off_street['Census year'] == 2023) &
        (off_street['Parking type'] == "Commercial") &
        (off_street['CLUE small area'].isin(["Melbourne (CBD)", "Docklands", "Southbank", "Carlton"]))
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

print(filtered_df.shape)
filtered_df.head()

(118, 11)


,Census year,Block ID,Property ID,Base property ID,Building address,CLUE small area,Parking type,Parking spaces,Longitude,Latitude,location
0,2023,735,110444,110444,100-110 Kings Way SOUTHBANK VIC 3006,Southbank,Commercial,15,144.961664,-37.827950,"-37.8279496529, 144.96166445403145"
1,2023,76,589841,105749,QV Retail & Parking 221 Little Lonsdale Street...,Melbourne (CBD),Commercial,1150,144.965725,-37.810679,"-37.81067861705, 144.965725053002"
2,2023,113,110685,108118,Queen Victoria Market Car Park Queen Victoria ...,Melbourne (CBD),Commercial,564,144.957627,-37.807833,"-37.807832574221266, 144.9576272223"
3,2023,115,104101,104101,58-64 Franklin Street MELBOURNE VIC 3000,Melbourne (CBD),Commercial,360,144.960816,-37.807417,"-37.80741705045211, 144.96081552815"
4,2023,241,573377,573377,Carpark Melbourne University 220 Berkeley Stre...,Carlton,Commercial,1136,144.960701,-37.800251,"-37.8002509946, 144.96070112606373"


In [37]:
# extract the postcode
filtered_df["postcode"] = filtered_df['Building address'].str.extract(r'(\d{4})$') 
filtered_df.head()

,Census year,Block ID,Property ID,Base property ID,Building address,CLUE small area,Parking type,Parking spaces,Longitude,Latitude,location,postcode
0,2023,735,110444,110444,100-110 Kings Way SOUTHBANK VIC 3006,Southbank,Commercial,15,144.961664,-37.827950,"-37.8279496529, 144.96166445403145",3006
1,2023,76,589841,105749,QV Retail & Parking 221 Little Lonsdale Street...,Melbourne (CBD),Commercial,1150,144.965725,-37.810679,"-37.81067861705, 144.965725053002",3000
2,2023,113,110685,108118,Queen Victoria Market Car Park Queen Victoria ...,Melbourne (CBD),Commercial,564,144.957627,-37.807833,"-37.807832574221266, 144.9576272223",3000
3,2023,115,104101,104101,58-64 Franklin Street MELBOURNE VIC 3000,Melbourne (CBD),Commercial,360,144.960816,-37.807417,"-37.80741705045211, 144.96081552815",3000
4,2023,241,573377,573377,Carpark Melbourne University 220 Berkeley Stre...,Carlton,Commercial,1136,144.960701,-37.800251,"-37.8002509946, 144.96070112606373",3053


In [43]:
filtered_df["postcode"].unique()

array(['3006', '3000', '3053', '3008'], dtype=object)

In [45]:
# standardize column names lowercase and replace spaces with underscores
filtered_df.columns = filtered_df.columns.str.lower().str.replace(" ", "_")

In [47]:
# Select only specific columns
selected_cols = [
    "block_id",
    "property_id",
    "base_property_id",
    "building_address",
    "postcode",
    "parking_spaces",
    "longitude",
    "latitude"
]
final_df = filtered_df[selected_cols]

In [49]:
final_df.head()

,block_id,property_id,base_property_id,building_address,postcode,parking_spaces,longitude,latitude
0,735,110444,110444,100-110 Kings Way SOUTHBANK VIC 3006,3006,15,144.961664,-37.827950
1,76,589841,105749,QV Retail & Parking 221 Little Lonsdale Street...,3000,1150,144.965725,-37.810679
2,113,110685,108118,Queen Victoria Market Car Park Queen Victoria ...,3000,564,144.957627,-37.807833
3,115,104101,104101,58-64 Franklin Street MELBOURNE VIC 3000,3000,360,144.960816,-37.807417
4,241,573377,573377,Carpark Melbourne University 220 Berkeley Stre...,3053,1136,144.960701,-37.800251


In [53]:
# save file as json and csv
final_df.to_json("off_street_parking.json", orient="records", indent=4)
final_df.to_csv("off_street_parking.csv", index=False)
print("Files saved: filtered_off_street_parking.json, filtered_off_street_parking.csv")

Files saved: filtered_off_street_parking.json, filtered_off_street_parking.csv
